In [14]:
import pandas as pd
from statistics import *
from math import isnan
import matplotlib.pyplot as plt # библиотека визуализации
from scipy import stats # библиотека для расчетов
from sklearn.metrics import matthews_corrcoef 
import seaborn as sns
import plotly
import plotly.express as px

В футболе статистические функции могут представлять собой хорошо известные модели, такие как модель регрессии Пуассона или модель Диксона и Коулза.

In [15]:
bundes_22_23 = pd.read_csv("data/bundes_22-23.csv")
bundes_21_22 = pd.read_csv("data/bundes_21-22.csv")
liga_1_22_23 = pd.read_csv("data/liga_1_22-23.csv")
liga_1_21_22 = pd.read_csv("data/liga_1_21-22.csv")
Prem_Leag_22_23 = pd.read_csv("data/Prem_Leag_22-23.csv")
Prem_Leag_21_22 = pd.read_csv("data/Prem_Leag_21-22.csv")
Premiere_22_23 = pd.read_csv("data/Premiere_22-23.csv")
Premiere_21_22 = pd.read_csv("data/Premiere_21-22.csv")
bundus_2020_2021 = pd.read_csv("data_auto/bundus_2020-2021.csv")
bundus_2019_2020 = pd.read_csv("data_auto/bundus_2019-2020.csv")

In [16]:
#df_b = pd.read_csv("data/bundes_22-23.csv")
#df_l = pd.read_csv("data/table_res_math_Prem_Leag_22-23.csv")
df = pd.concat([bundes_22_23,bundes_21_22,liga_1_22_23,liga_1_21_22,Prem_Leag_22_23,Prem_Leag_21_22,Premiere_22_23,Premiere_21_22,bundus_2020_2021,bundus_2019_2020])
df = df.drop('Unnamed: 0',axis=1)
df = df[df["счет1"] != '-']
df = df.reset_index().drop(['index'], axis = 1)

In [17]:
display(df.head())



,id,общая информация,Дата,Команда1,счет1,Команда2,счет2,Температура,Признак погоды,поб1,...,Офсайды (соперник) 1,Офсайды (соперник) 2,Желтые карточки 1,Желтые карточки 2,Желтые карточки (соперник) 1,Желтые карточки (соперник) 2,Красные карточки 1,Красные карточки 2,Красные карточки (соперник) 1,Красные карточки (соперник) 2
0,1736825,"Бундеслига, 1-й тур, регулярный сезон, 05.08.2...",05.08.2022 21:35,Айнтрахт Франкфурт,1,Бавария,6,+29°C,солнечно,6.50,...,2.00,2.5,0.88,1.90,1.00,1.50,0.0,0.10,0.25,0.0
1,1736826,"Бундеслига, 1-й тур, регулярный сезон, 06.08.2...",06.08.2022 19:30,Боруссия Дортмунд,1,Байер,0,+27°C,солнечно,2.13,...,1.63,1.0,0.75,0.89,1.25,1.00,0.0,0.00,0.00,0.0
2,1736827,"Бундеслига, 1-й тур, регулярный сезон, 06.08.2...",06.08.2022 16:30,Унион Берлин,3,Герта,1,+28°C,солнечно,1.87,...,0.67,2.0,1.20,1.75,0.80,0.75,0.0,0.25,0.00,0.0
3,1736828,"Бундеслига, 1-й тур, регулярный сезон, 07.08.2...",07.08.2022 18:30,Кёльн,3,Шальке,1,+30°C,солнечно,2.08,...,1.40,2.5,2.00,0.40,1.67,0.00,0.0,0.00,0.00,0.0
4,1736829,"Бундеслига, 1-й тур, регулярный сезон, 06.08.2...",06.08.2022 16:30,Боруссия М,3,Хоффенхайм,1,+27°C,солнечно,2.07,...,2.33,2.0,0.29,2.00,0.57,1.40,0.0,0.00,0.00,0.0


In [18]:
df = df.astype({'счет1': 'int32'})
df = df.astype({'счет2': 'int32'})

df["тотал"] = df["счет1"] + df["счет2"]
df["Дата"] = pd.to_datetime(df['Дата'],dayfirst=True)
df['Температура'] = df['Температура'].apply(lambda x: int(x[1:3]))

In [19]:
fig = px.imshow(
    df.corr(),
    text_auto=True,
    height=1400,
    width=1400
)
fig.show()

In [20]:
df["Признак погоды"].value_counts()

солнечно                                                    1508
переменная облачность                                       1322
пасмурно                                                     298
ясно                                                         233
небольшой дождь                                              123
облачно                                                       43
умеренный дождь                                               38
умеренный или сильный ливневый дождь                          36
в отдельных районах умеренный или сильный дождь с грозой      15
временами умеренный дождь                                     13
дымка                                                          6
Name: Признак погоды, dtype: int64

In [21]:
df[df["счет1"] == '-']["id"]

Series([], Name: id, dtype: int64)

In [22]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3635 entries, 0 to 3634
Data columns (total 73 columns):
 #   Column                          Non-Null Count  Dtype         
---  ------                          --------------  -----         
 0   id                              3635 non-null   int64         
 1   общая информация                3635 non-null   object        
 2   Дата                            3635 non-null   datetime64[ns]
 3   Команда1                        3635 non-null   object        
 4   счет1                           3635 non-null   int32         
 5   Команда2                        3635 non-null   object        
 6   счет2                           3635 non-null   int32         
 7   Температура                     3635 non-null   int64         
 8   Признак погоды                  3635 non-null   object        
 9   поб1                            3635 non-null   float64       
 10  X                               3635 non-null   float64       
 11  поб2

In [23]:
res_tab = df.groupby('Температура')['тотал'].mean().sort_values(ascending=False)
fig = px.bar(res_tab)
fig.show()

In [28]:
df["Отдых: дни (часы) 1"].value_counts()


(167) 7.0     476
(164) 6.8     100
(65) 2.7       92
(143) 6.0      90
(66) 2.8       83
             ... 
(382) 15.9      1
(319) 13.3      1
(402) 16.8      1
(365) 15.2      1
(375) 15.6      1
Name: Отдых: дни (часы) 1, Length: 246, dtype: int64

TypeError: unsupported operand type(s) for -: 'str' and 'str'